In [211]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [212]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Матвеичева"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [213]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [214]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [215]:
#pairplot(train_data)

In [216]:
# Определим размер валидационной выборки
val_size = round(0.15*len(train_data))
print(val_size)

49


In [217]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

280 49


In [218]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [219]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [220]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [221]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.000019,0.023813,0.231213,0.000103,0.152806,0.25
1,0.000012,0.023844,0.020374,0.149645,0.049442,0.50
2,0.000004,0.023812,0.038186,0.081357,0.028366,0.61
3,0.000053,0.023964,0.134975,0.002182,0.199482,0.57
4,0.000014,0.023832,0.140291,0.000189,0.063121,1.06
...,...,...,...,...,...,...
275,0.000012,0.023838,0.021984,0.149645,0.058444,0.47
276,0.005737,0.000000,0.231213,0.692601,0.008167,0.24
277,0.000210,0.049084,0.231213,0.149645,0.058312,0.31
278,0.000138,0.049084,0.231213,0.149645,0.030561,0.22


In [222]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [223]:
# Создадим простую модель логистической регрессии
model = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())

In [224]:
# Обучим модель
model.fit(x_train, y_train)

TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=MinMaxScaler())

In [225]:
# Проверим работу обученной модели на валидационной выборке
pred_val = model.predict(x_val)
mse = mean_squared_error(y_val, pred_val)
print(mse)

0.03967688120320564


In [226]:
pred_val

array([0.32417948, 0.33365696, 0.33452365, 0.33466462, 0.37255192,
       0.33262775, 0.32988709, 0.33292333, 0.33421256, 0.33350198,
       0.33378437, 0.43136885, 0.24441339, 0.33285467, 0.40435904,
       0.33336757, 0.33314701, 0.33324667, 0.33308787, 0.33370889,
       0.29040421, 0.2908312 , 0.37508467, 0.18286858, 0.33149309,
       0.18336492, 0.32993958, 0.33256944, 0.3574562 , 0.33060581,
       0.33442488, 0.39879003, 0.3349333 , 0.3692194 , 0.22182846,
       0.28982827, 0.33448021, 0.33431405, 0.33345569, 0.16939087,
       0.33528726, 0.43260411, 0.33450824, 0.35141943, 0.1633324 ,
       0.2878113 , 0.37845695, 0.32946489, 0.37472861])

In [227]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [228]:
test[y_column] = 0.0

In [229]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [230]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [231]:
x_test = sc_test[x_columns]

In [232]:
test[y_column] = model.predict(x_test)

In [233]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.216262
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.351436
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.378177
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.376188
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.332444


In [234]:
test.to_csv('task5_results/rus_stocks_funds_result.csv', index=False)